In [3]:
%%html
<style>

div.text_cell_render{
    font-size:14pt;
    }

</style>

In [4]:
import numpy as np
import scipy.linalg as la

### Modelling Circuit
<img src="circuit.png" style="width: 300px">

One source unit 0 taking the constant outside feed (10kg Gormanium, 100kg waste), <br>
<br>
The final concentration 4 and final tails 5 (They are not units!).

Each unit has exactly two output flows (**concentrates** and **tails**), and can have multiple input flows, we call the addition of all the input flows as the **feed** of each unit.

### Relationship of mass in discrete time steps 

We assume that all units taking in their feed and produce their outflows within a time step, that indicates for time step $n$, (we use $F_i$ to denote the feed for unit i):
<br>
<br>
<br>
\begin{align*}
\text{concentrate_outflow}_i^n (\text{Gormanium}) &= F_i^n (\text{Gormanium}) \cdot \text{recover_rate_Gormanium} \, (0.2), \\
\text{tails_outflow}_i^n (\text{Gormanium}) &= F_i^n (\text{Gormanium}) \cdot \left(1 - \text{recover_rate_Gormanium}\right) \\
\text{concentrate_outflow}_i^n (\text{waste}) &= F_i^n (\text{Gormanium}) \cdot \text{recover_rate_waste} \, (0.05), \\
\text{tails_outflow}_i^n (\text{waste}) &= F_i^n (\text{Gormanium}) \cdot \left(1 - \text{recover_rate_waste}\right) \\
\end{align*}


And in the next time step:
<br>
<br>
\begin{align*}
F_i^{n+1} (\text{Gormanium}) &= \sum_j \text{concentrate_outflow}_j^n (\text{Gormanium})\\
&+ \sum_k \text{tails_outflow}_k^n (\text{Gormanium})
\end{align*}
<br>
where conc_number of unit j is i, and tails_number of unit k is i.
<br>
<br>
Thus:
<br>
\begin{align*}
F_i^{n+1} (\text{Gormanium}) &= \sum_j F_j^n (\text{Gormanium}) \cdot     \text{recover_rate_Gormanium}\\
&+\sum_k F_k^n (\text{Gormanium}) \cdot     (1 - \text{recover_rate_Gormanium})
\end{align*}
<br>
where conc_number of unit j is i, and tails_number of unit k is i.
<br>
<br>

When mass-balance achieved, we have:
<br>
<br>
\begin{align*}
F_i^{n+1} (\text{Gormanium}) &= F_i^{n} (\text{Gormanium})\\
F_i^{n+1} (\text{Waste}) &=F_i^{n} (\text{Waste})
\end{align*}

for all units i and the Final Concs (4 in this circuit) and Final tails (5 in this circuit).

### Implment the iteration method
   Using this relationship, we implemented a iteration solver, as followed:
```c++
void CCircuit::calculate_monetary_value(bool &diverge, double tolerance, int max_iter)
{    
   // set the initial feed state
    clear();
    Feed.set(CCircuit::feed_mass_gormanium, CCircuit::feed_mass_waste);
    units[source].feed.set(Feed);
    
    // do the flow iteration
    while(tolerance condition)
    {
        // produce conc and tails for each units[i]
        for(int i = 0; i < num_units; ++i)
        {
            units[i].calculate_outflow();
        }

        // feed -> old_feed, feed = 0
        save_flow();

        // remember the source is always adding a constant Feed!!
        units[source].feed.set(Feed);
        
        // passing componants of conc and tails of units to the feed of their connectors
        for (int i = 0; i < num_units; ++i)
        {
            conc_num = units[i].conc_num;

            if (conc_num < num_units)
            {
                units[conc_num].feed.plus(units[i].conc);
            }
            else if(conc_num == num_units) Conc.plus(units[i].conc);
            else Tails.plus(units[i].conc);
            tails_num = units[i].tails_num;
            if (tails_num < num_units)
            {
                units[tails_num].feed.plus(units[i].tails);
            }
            if(tails_num == num_units) Conc.plus(units[i].tails);
            else Tails.plus(units[i].tails);
        }

        // some code to check diverge
        iter++;
    }
    if(not diverge)
    {
        monetary_value = CCircuit::value_gormanium * Conc.Gormanium
                         + CCircuit::value_waste * Conc.waste;
    }
    else
    {
        diverge = true;
        monetary_value = CCircuit::worst_case;
    }
}
```

### Express the relationship above using Matrix format
<br>
Recall we have:

<br>
\begin{align*}
F_i^{n+1} (\text{Gormanium}) &= \sum_j F_j^n (\text{Gormanium}) \cdot     \text{recover_rate_Gormanium}\\
&+\sum_k F_k^n (\text{Gormanium}) \cdot     (1 - \text{recover_rate_Gormanium})
\end{align*}

where conc_number of unit j is i, and tails_number of unit k is i.

<br>
we want to express all the $F_i^n$ (Gormanium), and $F_i^n$ (Waste) as a vector,
<br>

How many elements do we have? $2 \cdot \left(\text{num_units} + 2 \right)$ (adding Final Concs and tails)

$$
\boldsymbol{x}^n = \begin{pmatrix}
F_0^{n} (\text{Gormanium})\\
F_0^{n} (\text{waste})\\
F_1^{n} (\text{Gormanium})\\
F_1^{n} (\text{waste})\\
\vdots\\
F_{\text{FinalConc}}^{n} (\text{Gormanium})\\
F_{\text{FinalConc}}^{n} (\text{waste})\\
F_{\text{FinalTails}}^{n} (\text{Gormanium})\\
F_{\text{FinalTails}}^{n} (\text{waste})\\
\end{pmatrix}
$$
<br>

with Matrix A (very similar to a connectivity matrix) with:
<br>
<br>
$$ A_{ij}=\left\{
\begin{aligned}
\text{recover_rate_Gormanium}, \,\, \text{where} \, i = 2 \cdot \text{some unit number and  }  j \text{  is the conc_number of it} \\
\text{1 - recover_rate_Gormanium}, \,\, \text{where} \, i = 2 \cdot \text{some unit number and  }  j \text{  is the tails_number of it} \\
\text{recover_rate_waste}, \,\, \text{where} \, i = 2 \cdot \text{some unit number} + 1 \text{ and  }  j \text{  is the conc_number of it} \\
\text{1 - recover_rate_waste}, \,\, \text{where} \, i = 2 \cdot \text{some unit number} + 1 \text{ and  }  j \text{  is the tails_number of it}\\
0, \text{otherwise}
\end{aligned}
\right.
$$
<br>
remember the source is always adding a constant Feed, say $\boldsymbol{b}$, with 
<br>
<br>
$$ b_{i}=\left\{
\begin{aligned}
\text{feed_rate_Gormanium}, (10) \,\, \text{where} \, i = 2 \cdot \text{source_number} \\
\text{feed_rate_waste},  (100)\,\, \text{where} \, i = 2 \cdot \text{source_number} + 1\\
0, \text{otherwise}
\end{aligned}
\right.
$$

thus we could express the relationship (in matrix format) as 

$$
\boldsymbol{x}^{n+1} = A \left(\boldsymbol{x}^n + \boldsymbol{b}\right)
$$

and recall when mass-balance achieved, we have:
<br>
<br>
\begin{align*}
F_i^{n+1} (\text{Gormanium}) &= F_i^{n} (\text{Gormanium})\\
F_i^{n+1} (\text{Waste}) &=F_i^{n} (\text{Waste})
\end{align*}

for all units i and the Final Concs (4 in this circuit) and Final tails (5 in this circuit).

That is, 
$$
\boldsymbol{x}^{n+1} = \boldsymbol{x}^n  \,\,\,(\text{a fix point})
$$

We can just solve the fix point $\boldsymbol{x}$ by solving the system:
<br>
<br>
$$
\left(A - I\right)\,\boldsymbol{x} =  - A \, \boldsymbol{b}
$$

where $I$ is the Identity Matrix

### implement the direct solver
```c++
void CCircuit::calculate_monetary_value_directly()
{
    int n = CCircuit::cells + 3;

    auto A = double[n][n];
    auto b = double[n];
    
    // initialize A and b
    for (int i = 0; i < n; ++i)
    {   
        if (i == source * 2) b[i] = -CCircuit::feed_mass_gormanium;
        else if(i == source * 2 + 1) b[i] = -CCircuit::feed_mass_waste;
        else b[i] = 0;
        for(int j = 0; j < n; ++j) A[i][j] = 0;
    }
    
    // setting up A
    for (int i = 0; i < num_units; ++i)
    {              
        A[2 * units[i].conc_num][2 * i] = CCircuit::c_rate_gormanium;
        A[2 * units[i].conc_num + 1][2 * i + 1] = CCircuit::c_rate_waste;
        A[2 * units[i].tails_num][2 * i] = CCircuit::t_rate_gormanium;
        A[2 * units[i].tails_num + 1][2 * i + 1] = CCircuit::t_rate_waste;
    }

    matmul(A, b, n);

    for (int k = 0; k < n; ++k) 
        A[k][k]-= 1;

    Guassian_elimination(A, b, n);

    monetary_value = 100 * b[2 * num_units] - 500 * b[2 * num_units + 1];

}
```

### Compare the  iteration solver and the direct solver

In [15]:
conc_g_rate = 0.2
conc_w_rate = 0.05
feed_g = 10
feed_w = 100

The iteration solver write in python, with the same methodlogy.

In [16]:
def mass_balance_iterate(vec, conc_g_rate, conc_w_rate, feed_g, feed_w):
  
    tails_g_rate = 1 - conc_g_rate
    tails_w_rate = 1 - conc_w_rate
    units = int((len(vec) - 1) / 2)
    source = vec[0]
    num_elements = len(vec) + 3
    A = np.zeros((num_elements, num_elements))

    b = np.zeros(num_elements)
    res = 1.1
    b_new = b
    itera = 0

    conc_old = np.array(b[2 * units], b[2 * units + 1])

    for i in range(units):
        
        conc = vec[2 * i + 1]
        tails = vec[2 * i + 2]
        
        A[2 * conc][2 * i] = conc_g_rate
        A[2 * conc + 1][2 * i + 1] = conc_w_rate
        A[2 * tails][2 * i] = tails_g_rate
        A[2 * tails + 1][2 * i + 1] = tails_w_rate
    
    while(res > 1e-3):

        b[2 * source] = b[2 * source] + feed_g
        b[2 * source + 1] = b[2 * source + 1] + feed_w
        conc_old = (b[2 * units], b[2 * units + 1])
       
        b_new = A @ b
        conc_new = np.array([b_new[2 * units], b_new[2 * units + 1]])
        res = max(b_new - b)
        #print(conc_new)
        #print(100 * b_new[2 * units] - 500 * b_new[2 * units + 1])
        b = b_new
        conc_old = conc_new
        itera = itera + 1

    conc_g = b[2 * units]
    conc_w = b[2 * units + 1]

    return conc_g, conc_w, 100 * conc_g - 500 * conc_w

The direct solver write in python, with the same methodlogy.

In [17]:
def mass_balance_direct(vec, conc_g_rate, conc_w_rate, feed_g, feed_w):
  
    tails_g_rate = 1 - conc_g_rate
    tails_w_rate = 1 - conc_w_rate
    units = int((len(vec) - 1) / 2)
    source = vec[0]
    num_elements = len(vec) + 3
    A = np.zeros((num_elements, num_elements))

    b = np.zeros(num_elements)

    b[2 * source] = -feed_g
    b[2 * source + 1] = -feed_w    

    for i in range(units):
        
        conc = vec[2 * i + 1]
        tails = vec[2 * i + 2]
        
        A[2 * conc][2 * i] = conc_g_rate
        A[2 * conc + 1][2 * i + 1] = conc_w_rate
        A[2 * tails][2 * i] = tails_g_rate
        A[2 * tails + 1][2 * i + 1] = tails_w_rate

    b = A @ b

    C = A - np.eye(num_elements)
    
    b = la.solve(C, b)  
    conc_g = b[2 * units]
    conc_w = b[2 * units + 1]
    momentary_value = 100 * conc_g - 500 * conc_w

    print('conc_g: {0:f}, conc_w: {1:f}, momentory_value: {2:f}' .format(conc_g, conc_w, momentary_value))

See this example (10 units):

In [18]:
vec = np.array([7, 6,  1,  2,  4,  3,  5,  10,  6,  5, 11,  3,  7,  3,  2,  6,  8,  6,  9,  6,  0])

In [19]:
mass_balance_direct(vec, conc_g_rate, conc_w_rate, feed_g, feed_w)

conc_g: 2.927372, conc_w: 0.253969, momentory_value: 165.752738


In [20]:
mass_balance_iterate(vec, conc_g_rate, conc_w_rate, feed_g, feed_w)

(2.9254137218245777, 0.253968992775473, 165.5568757947213)

Also this example (10 units):

In [21]:
vec2 = np.array([4, 5, 4, 2, 3, 6, 1, 2, 4, 1, 0, 7, 9, 10, 9, 8, 1, 11, 0, 3, 4], dtype = int)

In [22]:
mass_balance_direct(vec2, conc_g_rate, conc_w_rate, feed_g, feed_w)

conc_g: 7.918093, conc_w: 70.731489, momentory_value: -34573.935285


In [23]:
mass_balance_iterate(vec2, conc_g_rate, conc_w_rate, feed_g, feed_w)

(7.9180927652541415, 70.7298763980363, -34573.12892249274)

For some test cases, the iteration solver would converge very very slow! Unless you code up an exit to mark it as diverge or you would wait it a couple of minutes to achieve the fix point.

### Final words for Circuit modelling
   <br>
   For a large system with more units, the computational cost to do matrix multiplications and Gaussian-Elimations would be very large. So the direct solver would be struggle to work out the performance for even one single circuit.
<br>
<br>
   However, The Base Case Circuit Specification in this porject (10 units) would not lead a high computational cost for our direct Matrix solver, so in exploring the best Circuit Configuration under the base case,
we would prefer using the direct Matrix solver rather than the iteration solver.

<br>
For larger systems, maybe a mixture of them.